In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import argparse
import os
import shutil
import time

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np

from datetime import datetime
import torchvision.models as models
import networks.load as load


import sys 
sys.path.append('..')
from utils_gsp.logger import Logger
# from utils_gsp import sps_tools
from gsp_model import GSP_Model
import utils_gsp.gsp_general as gsp_gen
import tools.ipynb_funcs as utilfuncs
import tools.visualization as viz

from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

from main import train, validate, accuracy, save_checkpoint, setup_experiment, gsp_sparse_training, AverageMeter

In [3]:
class Args:
    arch = 'resnet20'
    workers = 4
    epochs=160
    start_epoch=0
    batch_size = 128
    lr = 0.1
    momentum = 0.9
    weight_decay=1e-4
    print_freq = 50
    resume = False
    evaluate = False
    pretrained = False
    half = False
    exp_name = 'gsp_test'

    gpu=None
    logdir = '/logdir'
    gsp_training = True 
    gsp_sps = 0.8
    scheduled_sps_run = True
    proj_filters = False
    proj_model = False
    gsp_int = 150
    gsp_start_ep = -1
    finetune = False
    finetune_sps = 0.9


global args, best_acc1
args = Args
# writer = SummaryWriter()

In [4]:
# args.resume = './results/gspS0.9/gsp/model_best.pth.tar' # LOAD GSP MODEL

# args.resume = './results/gspS0.80/fine_0.9/model_best.pth.tar'
# args.finetune = True

In [42]:
# torch.manual_seed(0)
# Setup the experiment
flogger = setup_experiment(args)
args.logger.log_cmd_arguments(args)

# model = torch.nn.DataParallel(resnet.__dict__[args.arch]())
# model = VGG('D')
model = load.model(args.arch)
model.cuda()

model_gsp = GSP_Model(model)

if args.finetune:
    flogger.info(15*"*" + " Model will be finetuned!! " + 15*"*")
    model_gsp.prune_and_mask_model(sps=args.finetune_sps)
    
# optionally resume from a checkpoint
if args.resume:
    if os.path.isfile(args.resume):
        print("=> loading checkpoint '{}'".format(args.resume))
        checkpoint = torch.load(args.resume)
        args.start_epoch = checkpoint['epoch']
        best_acc1 = checkpoint['best_acc1']
        model_gsp.model.load_state_dict(checkpoint['state_dict'])
        print("=> loaded checkpoint '{}' (epoch {})"
                .format(args.evaluate, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(args.resume))

cudnn.benchmark = True

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])

train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root='./data', train=True, transform=transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, 4),
        transforms.ToTensor(),
        normalize,
    ]), download=True),
    batch_size=args.batch_size, shuffle=True,
    num_workers=args.workers, pin_memory=True)

val_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root='./data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ])),
    batch_size=128, shuffle=False,
    num_workers=args.workers, pin_memory=True)


# ----------------------- Make a GSP Model -----------------------
print(f"The sparsity of the model is: {model_gsp.get_model_sps():.2f}")
args.writer = SummaryWriter(log_dir=f'results/{args.exp_name}/runs/{datetime.now().strftime("%m-%d_%H:%M")}')

# define loss function (criterion) and optimizer
criterion = nn.CrossEntropyLoss().cuda()

optimizer = torch.optim.SGD(model.parameters(), args.lr,
                            momentum=args.momentum,
                            weight_decay=args.weight_decay)

# ============================ Setup GSP model ============================
if args.gsp_training:
    gsp_sparse_training(model_gsp, train_loader, args)
    flogger.info(15*"*" + " Model will be trained with GSP Sparsity!! " + 15*"*" )


 input logdir: ./results/gsp_test/logdir 


 RETURNED FILE LOGGER! 

Files already downloaded and verified
The sparsity of the model is: 0.26


In [6]:
# for name, layer in model.named_modules():
#     if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
#         print(f"Name: {name} | shape: {layer.weight.shape}")

In [43]:
names, weight_l, shape_l, weight_d, layers_d = utilfuncs.get_model_layers(model)

In [44]:
def random_prune(self, sps=0.95):
    for name, layer in model_gsp.model.named_modules():
        if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
            num = layer.weight.data.numel()
            num_zero = int(round(num*sps))
            indices = torch.randperm(num)[:num_zero]
            layer.weight.data.flatten()[[indices]] = 0.
            # print(layer.weight.shape)

In [45]:
model_gsp.print_model_sps()

Model Sparsity: 89.79 %


In [24]:
mytensor = weight_d[names[0]]

n = mytensor.numel()
m = int(round(n*0.6))
indices = torch.randperm(n)[:m]



In [30]:
mytensor.flatten()[[indices]] = 0.

In [31]:
mytensor

tensor([[[[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 1.0591e-01,  0.0000e+00, -3.9859e-01],
          [ 0.0000e+00,  2.1624e-01,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00, -1.6816e-01],
          [-1.5996e-01,  0.0000e+00,  0.0000e+00],
          [-1.6005e-01,  0.0000e+00,  5.7319e-01]],

         [[ 0.0000e+00,  0.0000e+00,  3.3461e-01],
          [-7.7171e-02,  0.0000e+00,  0.0000e+00],
          [ 9.4491e-02, -1.6816e-01, -4.3890e-01]]],


        [[[ 0.0000e+00,  0.0000e+00,  1.2645e-01],
          [ 0.0000e+00, -1.8461e-02, -3.2836e-01],
          [ 0.0000e+00,  0.0000e+00, -3.5386e-01]],

         [[ 0.0000e+00,  2.6042e-01,  1.6740e-01],
          [ 0.0000e+00,  0.0000e+00,  5.7339e-02],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  6.8927e-02,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00]]],


        [[[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [-2.2

In [7]:
model_gsp.scheduled_sps_run = True
model_gsp.total_epochs = 10

In [13]:
args.epoch = 10
train(train_loader, model_gsp, criterion, optimizer, 0, args, gsp_mode=True)
# validate(val_loader, model, criterion, args)

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


Projecting layers with GSP!! | GSP_Mode: True
Projecting layers with GSP!! | GSP_Mode: True
Projecting layers with GSP!! | GSP_Mode: True


In [ ]:
model_gsp.get_model_sps()
model_gsp.print_model_sps()
model_gsp.proj_filters=True

In [ ]:
# model_gsp.prune_and_mask_model(sps=0.9)

## Global GSP

In [ ]:
def project_all_layers(self, sps):
    layer_d = dict()
    shape_d = dict()
    ctr = 0
    for name, layer in model.named_modules():
        if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
            shape_d[name] = layer.weight.shape
            layer_d[ctr] = layer.weight.data.detach().clone().flatten()
            ctr+=1

    xp_mat, ni_list = gsp_gen.GSP(layer_d, sps=sps)
    out_layers = gsp_gen.unpad_output_mat(xp_mat, ni_list)

    # rebuild_network
    ctr = 0
    for name, layer in model.named_modules():
        if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
            layer.weight.data = out_layers[ctr].reshape(shape_d[name])
            ctr += 1

In [ ]:
project_all_layers(model_gsp, sps=0.85)

In [ ]:
def padded_sparsity(matrix, ni_list):
    """
    This Hoyer Sparsity Calculation is for matrices with the end of the columns that are padded. Hence,
    it needs the information of how much of each columns are elements and how much of them are padded.
    ni_list: Contains the number of values in each column (rest are padded with zero).
    """

    ni = matrix.shape[0]
    device = matrix.device
    ni_tensor = torch.tensor(ni_list).to(device)

    # Get Indices of all zero vector columns.
    zero_col_ind = (abs(matrix.sum(0) - 0) < 2.22e-16).nonzero().view(-1)  
    spx_c = (torch.sqrt(ni_tensor) - torch.norm(matrix,1, dim=0) / torch.norm(matrix,2, dim=0)) / (torch.sqrt(ni_tensor) - 1)
    if len(zero_col_ind) != 0:
        spx_c[zero_col_ind] = 1  # Sparsity = 1 if column already zero vector.
    
    if matrix.dim() > 1:   
        sps_avg = spx_c.mean()
    elif matrix.dim() == 1:  # If not a matrix but a column vector!
        sps_avg =  spx_c    
    return sps_avg

## MODEL VISUALIZATION

In [ ]:
names, weight_l, shape_l, weight_d, layers_d = utilfuncs.get_model_layers(model)

In [ ]:
layers_d[names[0]].weight.data.detach().device

In [ ]:
# viz.plot_weight_dist(model, title='Resnet20 Distribution', arch='resnet20', 
#                     filename='./results/gspS0.80/fine_0.9/w_dist.png')

In [ ]:
def apply_gsp_prune(model_gsp, sps=0.8):
    model_gsp.force_apply_gsp(sps)
    print(f"SPS after gsp application: {model_gsp.get_model_sps():.2f}")
    
    model_gsp.prune_and_mask_model(sps)
    print(f"SPS after pruning: {model_gsp.get_model_sps():.2f}")


In [ ]:
apply_gsp_prune(model_gsp, sps=0.9)

## Randomg Testing

In [ ]:
# a = torch.ones((3,3,2,2))
# a[0,1,:,:] = 2
# a[0,2,:,:] = 3

# a[1,0,:,:] = 4
# a[1,1,:,:] = 5
# a[1,2,:,:] = 6

# a[2,0,:,:] = 7
# a[2,1,:,:] = 8
# a[2,2,:,:] = 9